In [18]:
import numpy as np
import random

The following Python function will create a string  representation of a Fortran expression for a polynomial.

In [33]:
def create_poly_expression(constant, *coeffs):
    expr = f'({constant})'
    for i, coeff in enumerate(coeffs):
        expr += f' &\n              + ({coeff})*x**{i + 1}'
    return expr

For example, we can now create a polynomial of degree 2.

In [34]:
print(create_poly_expression(1.0, 3.0, -5.0))

(1.0) &
              + (3.0)*x**1 &
              + (-5.0)*x**2


The following function will return a Fortran function definition for given coefficients of the polynomial.

In [35]:
def create_poly_function(name, constant, *coeffs):
    expr = create_poly_expression(constant, *coeffs)
    return f'''
    function {name}(x) result(y)
        implicit none
        real, value :: x
        real :: y
        
        y = {expr}
    end function {name}
    '''

In [36]:
print(create_poly_function('func1', 1.2, 3.5, -2.1))


    function func1(x) result(y)
        implicit none
        real, value :: x
        real :: y
        
        y = (1.2) &
              + (3.5)*x**1 &
              + (-2.1)*x**2
    end function func1
    


The next function will create a random polynomial with a given name.

In [37]:
def create_random_poly_function(name):
    degree = random.randint(0, 10)
    coeffs = np.random.normal(size=degree + 1)
    return create_poly_function(name, coeffs[0], *coeffs[1:])

In [38]:
print(create_random_poly_function('func1'))


    function func1(x) result(y)
        implicit none
        real, value :: x
        real :: y
        
        y = (1.4265728749067677) &
              + (-1.2457570237626667)*x**1 &
              + (1.3201876548848241)*x**2 &
              + (-0.8790831303443152)*x**3 &
              + (0.8247735882571433)*x**4 &
              + (-0.4385351609008346)*x**5 &
              + (0.4112613698619335)*x**6 &
              + (-0.6587906293050799)*x**7 &
              + (-1.0842532329414618)*x**8 &
              + (0.08736515053718635)*x**9
    end function func1
    


Now we create a Fortran source file with 5 random polynomial functions.

In [39]:
with open('functions.f90', 'w') as src_file:
    for i in range(5):
        print(create_random_poly_function(f'func{i + 1:03d}'), file=src_file)

Using f2py, this Fortran file can be converted into a Python module.

In [40]:
!f2py -c -m functions functions.f90

running build
running config_cc
unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
build_src
building extension "functions" sources
f2py options: []
f2py:> /tmp/tmpdgrbvska/src.linux-x86_64-3.8/functionsmodule.c
creating /tmp/tmpdgrbvska/src.linux-x86_64-3.8
Reading fortran codes...
	Reading file 'functions.f90' (format:free)
Post-processing...
	Block: functions
			Block: func001
			Block: func002
			Block: func003
			Block: func004
			Block: func005
Post-processing (stage 2)...
Building modules...
	Building module "functions"...
		Creating wrapper for Fortran function "func001"("func001")...
		Constructing wrapper function "func001"...
		  y = func001(x)
		Creating wrapper for Fortran function "func002"("func002")...
		Constructing wrapper function "func002"...
		  y = func002(x)
		Creating wrapper for Fortran function "func003"("fun

If everything went well, f2py has created a shared object that contains the functions, and a module that we can import.

In [41]:
!ls

f2py.ipynb  functions.cpython-38-x86_64-linux-gnu.so  functions.f90


In [42]:
import functions

The module contains the functions that were generated.

In [43]:
dir(functions)

['__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__version__',
 '_functions_error',
 'func001',
 'func002',
 'func003',
 'func004',
 'func005']

Note that some bare-bones documentation has been generated automatically.

In [44]:
?functions.func001

Call signature: functions.func001(*args, **kwargs)
Type:           fortran
String form:    <fortran func001>
Docstring:     
y = func001(x)

Wrapper for ``func001``.

Parameters
----------
x : input float

Returns
-------
y : float


We can collect the function names from the module and store the actual functions in a list.

In [46]:
funcs = list()
for name in dir(functions):
    if not name.startswith('_'):
        funcs.append(getattr(functions, name))

In [49]:
funcs[1](5.3)

1.4554624557495117

Now we can call the functions on some value of $x$.

In [51]:
x = 1.2
for func in funcs:
    print(f'{func.__name__}({x}) = {func(x)}')

func001(1.2) = -10.370649337768555
func002(1.2) = 1.4554624557495117
func003(1.2) = -1.170980453491211
func004(1.2) = -7.09548282623291
func005(1.2) = 5.6847968101501465
